## test table 생성

- 문제정의

웹상에 존재하는 이름 성별 CSV 파일을 Redshift에 있는 테이블로 복사

DROP TABLE IF EXISTS name_gender;<br>
CREATE TABLE name_gender (<br>
&nbsp;&nbsp;&nbsp;   name varchar(32),<br>
&nbsp;&nbsp;&nbsp;   gender varchar(8)<br>
);

In [3]:
from config import REDSHIFT_INFO, SCHEMA, DATA_LINK
import psycopg2


def get_redshift_connection(dbinfo):
    conn = psycopg2.connect(**dbinfo)
    conn.set_session(autocommit=True)
    return conn

# Assignment #1 - Colab Python 코드 개선하기
1. 헤더가 레코드로 추가되는 문제 해결하기
2. Idempotent하게 잡을 만들기 (full refresh잡이라고 가정)
    - 여러 번 실행해도 동일한 결과가 나오게 만들기


3. (Optional) Transaction을 사용해보기
    - BEGIN; DELETE FROM ..; INSERT INTO… ;END;


## 1. 헤더가 레코드로 추가되는 문제 해결하기

transform에 헤더검사 및 제거하는 조건문 추가


- 모든 `name_gender.csv` 파일이 항상 헤더가 고정되었다면, 인덱싱을 통해 전처리 (v)
- "name,gender" 문자열 제거

In [58]:
import requests

def extract(url):
    f = requests.get(url)
    return (f.text)


def transform(text):
    lines = text.split("\n")[1:]  # header 제외
    data = [tuple(line.split(",")) for line in lines if line != ""]  # 문자열 split, 빈 문자열 제거
    return list(set(data))  # 중복레코드 제거


# 데이터가 많아지면, copy 명령어를 통해 파일로 S3 등에 bulk하여 redshift에 적재
def load(dbinfo, SCHEMA, lines):
    # BEGIN과 END를 사용해서 SQL 결과를 트랜잭션으로 만들어주는 것이 좋음
    # BEGIN;DELETE FROM (본인의스키마).name_gender;INSERT INTO TABLE VALUES ('KEEYONG', 'MALE');....;END;
    conn = get_redshift_connection(dbinfo)
    cur = conn.cursor()
    try:
        for r in lines:
            insert_sql = f"""
            begin;
                delete from {SCHEMA}.name_gender where name = '{r[0]}' and gender = '{r[1]}';
                insert into {SCHEMA}.name_gender VALUES ('{r[0]}', '{r[1]}');
            end;
            """
            # insert_sql = f"insert into {SCHEMA}.name_gender VALUES (%s, %s); end;"
            # cur.executemany(insert_sql, lines)
            cur.execute(insert_sql)
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
    finally:
        conn.commit()
        conn.close()


def fetchall(dbinfo, schema_table, cols="*", option=None):
    conn = get_redshift_connection(dbinfo)
    cur = conn.cursor()
    
    # truncate {schema_table};
    sql = f"""
    select {cols} from {schema_table} {option};
    """
    cur.execute(sql)
    data = cur.fetchall()
    return data



## Extract - Transform - Load

In [62]:
data = extract(DATA_LINK)
lines = transform(data)
# lines.insert(3,("test", "test over 8 varchar"))
load(REDSHIFT_INFO, SCHEMA, lines)
result = fetchall(REDSHIFT_INFO, SCHEMA+".name_gender")

In [64]:
len(result)

100